<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Fundus_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential

In [0]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
train_path = "/content/drive/My Drive/Eye Disease Research/train"
test_path = "/content/drive/My Drive/Eye Disease Research/test"

imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.10)
train_batches = imgdata.flow_from_directory(train_path, subset='training', target_size=(512, 496) ,
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(512, 496) ,
                                           classes = ['CNV','DRUSEN','NORMAL'], batch_size = 16)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(512, 496) ,
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)


Found 64937 images belonging to 3 classes.
Found 751 images belonging to 3 classes.
Found 7214 images belonging to 3 classes.


In [64]:
print(train_batches.classes)

[0 0 0 ... 2 2 2]


In [95]:
vgg16_model = VGG16()
#vgg16_model.layers.pop()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [96]:
vgg16_model = VGG16(include_top = False, input_shape=(512,496,3), input_tensor=Input(shape=(512, 496, 3)))
#vgg16_model.layers.pop()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 512, 496, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 496, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 496, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 248, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 248, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 248, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 124, 128)     0     

In [100]:
model= Sequential()
for layer in (vgg16_model.layers):
  model.add(layer)
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 512, 496, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 496, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 248, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 248, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 248, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 124, 128)     0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 128, 124, 256)   

In [101]:
i = 1
for layer in model.layers:
  if(i%4 > 0):
    layer.trainable = False
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 512, 496, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 496, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 248, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 248, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 248, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 124, 128)     0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 128, 124, 256)   

In [0]:
model.compile(tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9, nesterov=False, name="SGD"), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), train_batches.classes)

In [108]:
model.fit_generator(train_batches, steps_per_epoch=8, epochs=16,verbose=1, 
                    validation_data=valid_batches, validation_steps=8, shuffle=True, class_weight = class_weights)

Epoch 1/16
8/8 [==============================] - 108s 13s/step - loss: 0.9671 - accuracy: 0.5469 - val_loss: 0.9175 - val_accuracy: 0.5352
Epoch 2/16
8/8 [==============================] - 120s 15s/step - loss: 0.9788 - accuracy: 0.4922 - val_loss: 1.0123 - val_accuracy: 0.4844
Epoch 3/16
8/8 [==============================] - 118s 15s/step - loss: 0.9594 - accuracy: 0.5469 - val_loss: 0.9050 - val_accuracy: 0.5508
Epoch 4/16
8/8 [==============================] - 120s 15s/step - loss: 0.9353 - accuracy: 0.5586 - val_loss: 0.9762 - val_accuracy: 0.5117
Epoch 5/16
8/8 [==============================] - 103s 13s/step - loss: 0.9345 - accuracy: 0.5703 - val_loss: 0.8912 - val_accuracy: 0.5312
Epoch 6/16
8/8 [==============================] - 105s 13s/step - loss: 0.9507 - accuracy: 0.5586 - val_loss: 0.9642 - val_accuracy: 0.5039
Epoch 7/16
8/8 [==============================] - 115s 14s/step - loss: 1.0228 - accuracy: 0.4492 - val_loss: 0.9635 - val_accuracy: 0.5273
Epoch 8/16
8/8 [====